Algorytm generowania danych:
1. Tworzę jeden zbiór ze wszystkich dotychczasowych scrapowań
2. Tworzę tabelę statystyk dla każdego kraju z podziałem na grupy dni (typu 0-4, 5-9 itd.), w tym celu wykonuję grupowanie i agregacje danych. Nie biorę w tym punkcie pod uwagę daty scrapowania. Jako wynik tego punktu otrzymyję tabelę, w której zebrane są średnia i wariancja ceny lotów na danej lini kraj-kraj, w zależności od przedziału czasu od odlotu do wylotu z podziałem na czas scrapowania.
3. Wracam do pliku z 2015-05-15.txt czyli pierwszego pliku scrapowania i biorę go jako plik referencyjny.
4. Przechodzę po każdym wierszu tego pliku i sprawdzam następujące pola:
['Country_from', 'Country_to', 'Days', 'Scrap_time']
5. Zmieniam datę scrapowania na dzień wcześniej, Days = Days + 1. Jeżeli days > 90 to resetuję rekord do tego samego z wyprzedzeniem losowanym z przedziału (0, 10).
6. Randomizuje godziny odlotów, przy czym z zachowaniem odpowiednich długości lotów. Tzn. robię tak: jak miałem lot morning - afternoon to roznica jest 1. Zatem losuje wylot na np. evening to powrot bede mial night
7. Na podstawie tabeli ze statystykami generuję cenę z rozkładu normalnego o podanej w tabeli średniej i warinacji. Updatuje rekord o nową cenę do której dodaję jeszcze całkiem losowy szum z rozkładu normalnego. 

Reszty predyktorów nie zmieniam. Można też robić jakoś tak, że generować te ceny cząstkowe, ale wydaje mi się, że żeby zachować te kolumny to trzeba policzyc taki sredni udzial tych cen.

In [ ]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import udf, lit
from pyspark.sql.types import *#IntegerType, StringType
import pyspark.sql.functions as F
import json
import itertools
import math
from datetime import datetime
import pandas as pd
import copy
import numpy as np
import matplotlib.pyplot as plt
import random

conf = SparkConf().setAppName('MyFirstStandaloneApp')
conf.set("spark.speculation","false")
sc = SparkContext(conf=conf)

#sc.setLogLevel("ERROR")

In [2]:
@udf
def groupped_days(days, bin_size):
    bin_size = int(bin_size)
    low = days // bin_size * bin_size
    up = low + bin_size - 1
    return(str(low) + '-' + str(up))

Ustalam szerokosc przedzialu po ktorym bede grupowal dni. Jak ustawie na 5, to bede mial grupy 0-4, 5-9, itd.

In [3]:
bin_size = 5

In [4]:
filepath = "Structured_data2"

flightsRDD = sc.textFile(filepath)
header = flightsRDD.first()
flightsRDD = flightsRDD.filter(lambda line : line != header)
colnames = header.split(';')
fields = [StructField(field_name, StringType(), True) for field_name in colnames]
schema = StructType(fields)
parts = flightsRDD.map(lambda line: line.split(';'))
sqlContext = SQLContext(sc)
dt = sqlContext.createDataFrame(parts, schema)
to_cast_int = ["Flight_id", "Days", "Journey_time"]
to_cast_double = ["Price1_There", "Price1_Back", "Price2_There", "Price2_Back", "Full_Price"]
for field in to_cast_int:
    dt = dt.withColumn(field,  dt[field].cast(IntegerType()))
for field in to_cast_double:
    dt = dt.withColumn(field,  dt[field].cast(DoubleType()))
dt.registerTempTable("flights")

df = dt

Tutaj tworzę takie pomocnicze kolumny, do dalszej obróbki. Robię też zmienną weekDay, która oznacza numer dnia tygodnia, ale nie korzystam z tego ostatecznie w żaden sposób. (Uwaga. 0 - niedziela, 1-poniedzialek itd.)

In [5]:
weekDay =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w'))

df = dt.withColumn('weekDay', weekDay(dt['Scrap_date']))
df = df.withColumn('grouppedDays', groupped_days(df['Days'], lit(bin_size)))
fun = udf(lambda x : int(x.split('-')[0]))
df = df.withColumn('startingBin', fun(df.grouppedDays))
#df = df.withColumn('daysToLeave', df['Days'] + df.['Journey_time'])

Tutaj tworzenie tabel statystycznych do generowania danych:

In [6]:
df1 = df.groupby(['Scrap_time', 'Country_from', 'Country_to', 'startingBin']).agg(
F.mean(df.Full_Price).alias('mean'),
F.stddev(df.Full_Price).alias('stdev'),
F.count(df.Full_Price).alias('count'))

df2 = df.groupby(['Scrap_time', 'Country_from', 'Country_to', 'Days']).agg(
F.mean(df.Full_Price).alias('mean'),
F.stddev(df.Full_Price).alias('stdev'),
F.count(df.Full_Price).alias('count'))

days_stats = df2.toPandas()
bin_stats = df1.toPandas()

Uporządkowanie typów w dataframe'ach:

In [7]:
def exists(obj, datastructure):
    return any([obj == elem for elem in datastructure])

In [8]:
asint = ['Scrap_time', 'startingBin', 'count', 'Days']
asfloat = ['mean', 'stdev']
day_col = list(days_stats.columns)
bin_col = list(bin_stats.columns)

for c1, c2 in zip(day_col, bin_col):
    if exists(c1 , day_col):
        if exists(c1, asint):
            days_stats[c1] = days_stats[c1].astype('int')
        elif exists(c1, asfloat):
            days_stats[c1] = days_stats[c1].astype('float')
    if exists(c2 , bin_col):
        if exists(c2, asint):
            bin_stats[c2] = bin_stats[c2].astype('int')
        elif exists(c2, asfloat):
            bin_stats[c2] = bin_stats[c2].astype('float')

Tutaj właściwe generowanie:

Wczytanie pliku referencyjnego, czyli pliku z najstarszego scrapowania: 2019-05-15.txt:

In [9]:
filepath = "Reference_file"

flightsRDD = sc.textFile(filepath)
header = flightsRDD.first()
flightsRDD = flightsRDD.filter(lambda line : line != header)
colnames = header.split(';')
fields = [StructField(field_name, StringType(), True) for field_name in colnames]
schema = StructType(fields)
parts = flightsRDD.map(lambda line: line.split(';'))
sqlContext = SQLContext(sc)
dt = sqlContext.createDataFrame(parts, schema)
to_cast_int = ["Flight_id", "Days", "Journey_time"]
to_cast_double = ["Price1_There", "Price1_Back", "Price2_There", "Price2_Back", "Full_Price"]
for field in to_cast_int:
    dt = dt.withColumn(field,  dt[field].cast(IntegerType()))
for field in to_cast_double:
    dt = dt.withColumn(field,  dt[field].cast(DoubleType()))
dt.registerTempTable("flights")

df = dt

Cofam się z datą o 1

In [10]:
df = df.withColumn('Scrap_date',  F.date_add(df['Scrap_date'], -1))

Jak się cofam z datą, to mi liczba dni wzrasta o 1

In [11]:
df = df.withColumn('Days', df['Days'] + 1)

Randomizuje predyktory mówiące o czasach odlotów i przylotów

In [12]:
# Randomizacja predyktorów
d = {'morning' : 0, 'afternoon' : 1, 'evening': 2, 'night': 3}
inv_d = dict((v, k) for k, v in d.items())

departs = ['Depart_hour1_There', 'Depart_hour2_There', 'Depart_hour1_Back', 'Depart_hour2_Back']
arrives = ['Arrival_hour1_There', 'Arrival_hour2_There', 'Arrival_hour1_Back', 'Arrival_hour2_Back']

@udf
def random_num():
    return random.randint(0, 3)

@udf
def subtr(depart, arrive):
    global d
    if (depart != 'none'):
        return abs(d[arrive] - d[depart])
    else:
        return 10
    
@udf
def update_time_depart(randInt):
    global inv_d
    return inv_d[randInt]

@udf
def update_time_arrive(randInt, subtr):
    global inv_d
    return inv_d[(int(randInt) + int(subtr)) % 4]

In [13]:
for depart, arrive in zip(departs, arrives):
    df = df.withColumn('randInt', random_num())
    df = df.withColumn('subtr', subtr(df[depart] , df[arrive]))
    df = df.withColumn(depart, F.when(df[depart] != 'none', update_time_depart(df['randInt'])).otherwise('none'))
    df = df.withColumn(arrive, F.when(df[arrive] != 'none', update_time_arrive(df['randInt'], df['subtr'])).otherwise('none'))

In [14]:
df = df.drop('randInt')
df = df.drop('subtr')

Updatuje cenę wg następującego algorytmu:
1. Sprawdzam czy liczba dni jest > 90, jezeli tak to zakładam ze ten lot sie pojawil w momencie odlotu samolotu z tej samej linii lub wyprzedania biltetów. Losowo losuję days z przedziału zatem (0,10)
2. Szukam w tabeli ze statystykami pasujących rekordów wg Country_from, Country_to oraz Scrap_time
3. Szukam w tabeli rekord, który ma najbardziej zbliżone wyprzedzenie do zadanego
4. Na podstawie tej tego rekordu, losuję z rozkładu normalnego o zadanej wg rekordu średniej i wariancji cenę
5. Dodaję do niej losowy szum
6. Zwracam wartosc ceny

In [15]:
datafr = df.toPandas()

In [16]:
def days_update(days):
    if days > 90:
        return random.randint(0, 10)
    else:
        return days

In [17]:
def update_price(full_price, days, country_from, country_to, scrap_time, bin_size):
    global days_stats
    global bin_stats
    bin_size = int(bin_size)
    records = bin_stats[(bin_stats.Country_from == country_from) & (bin_stats.Country_to == country_to) & 
                       (bin_stats.Scrap_time == int(scrap_time))]
    bucket = (days // bin_size) * bin_size
    array = np.array(records['startingBin'].tolist())
    array = np.asarray(array)
    idx = (np.abs(array - bucket)).argmin()
    bucket = array[idx]
    record = records[records.startingBin == bucket]
    # Tutaj chodzi o to, że moze wystapic jakas unikalna wartosc i wtedy nie bedzie dla niej wariancji, wtedy
    # Ustalam wariancje na 1
    if np.isnan(record['stdev'].item()) == False:
        new_price = np.random.normal(loc = record['mean'], scale = record['stdev'], size = 1)[0]
    else:
        new_price = np.random.normal(loc = record['mean'], scale = 1, size = 1)[0]
    new_price = new_price + np.random.rand()
    return new_price.tolist()

In [18]:
datafr.to_csv('test2-2019-05-14', sep = ';', header = True, index= False)

In [19]:
from tqdm import tqdm

tqdm.pandas()
datafr = pd.read_csv('test2-2019-05-14', sep = ';')
datafr['Days'] = datafr.progress_apply(lambda row : random.randint(0, 10) if (int(row[5]) > 90) else int(row[5]) , axis = 1 )

100%|███████████████████████████████████████████████████████████████████████| 136397/136397 [00:02<00:00, 47342.58it/s]


In [20]:
datafr['Full_price'] = datafr.progress_apply(lambda row: update_price(row[-1], row[5], row[2], row[3], row[1], bin_size), axis = 1)

100%|█████████████████████████████████████████████████████████████████████████| 136397/136397 [13:17<00:00, 171.42it/s]


In [21]:
datafr['Full_price'] = np.round(datafr['Full_price'], 2)

In [22]:
datafr.to_csv('test2-2019-05-14', sep = ';', header = True, index= False)

# Koniec